In [30]:

'''
this refactor is designed to build in
alternating player using two user defined agents

some templates are provided as well as a bot that uses heuristics to try to win


to solve 
'''

'\nthis refactor is designed to build in\nalternating player using two user defined agents\n\nsome templates are provided as well as a bot that uses heuristics to try to win\n\n\nto solve \n'

In [31]:
# uttt base class
import numpy as np
import matplotlib.pyplot as plt
from time import perf_counter
from tqdm import tqdm

''' visualization imports '''
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('mycmap', ['lightgrey', 'white'])
import matplotlib.colors as mcolors
tab10_names = list(mcolors.TABLEAU_COLORS) # create a list of colours

def checkerboard(shape):
    # from https://stackoverflow.com/questions/2169478/how-to-make-a-checkerboard-in-numpy
    return np.indices(shape).sum(axis=0) % 2



### Bot template
This is the format we're expecting for a UTTT bot.
It will have a name parameter and a .move method which takes a dictionary as input.


In [32]:
class bot_template:
    ''' 
    demonstrates the two minimal functions for an agent
    '''
    def __init__(self, name: str):
        self.name = name
        
    def move(self, board_dict: dict) -> tuple:
        '''
        the keywords for board_dict are:
        board_state: a 9x9 np.array showing which squares have which markers.
                     your markers at +1 and your opponent markers are -1.
                     open squares are 0
        active_box:  the coordinate for the active mini-board (indicates which 3x3 is currently playable)
                     a value of (-1, -1) is used if the whole board is valid
        valid_moves: a list of tuples indicating which positions are valid (in the 9x9 format)
        '''
        pass    
    

### Updated game engine
refactored to simplify running bots

In [56]:
class uttt_engine_agentic_refactor():
    def __init__(self):
        self.active_box = (-1,-1)    # (-1,-1) means any box can be played in
        self.board_state = np.zeros((9,9))
        self.finished_boxes = np.zeros((3,3)) # 1 for agent1, -1 for agent2. "6" indicates stalemate
        self.finished = False
        self.finished_win = False
        self.finished_stale = False
        self.current_player = 1 # starting player
        self.game_log = list() # < subject to change based on visualization interface

    def query_player(self) -> None:
        '''
        send a request to a player instance for a move
        updates the game board with the desired move
        if no valid move is returned, a random move is played for that agent
        '''
        # check agents are loaded
        if not hasattr(self, 'agents'):
            print('must load agents')
            return 
        
        # check game is not finished
        if self.finished:
            print('no valid moves in terminal state')
            print("(something went wrong, you shouldn't be here)")
            return
        
        # send the request in the form of a dictionary
        temp_valid_moves = self.get_valid_moves()
        query_dict = {'board_state': self.board_state * self.current_player,
                      'active_box': self.active_box,
                      'valid_moves': temp_valid_moves}

        try:
            desired_move = tuple(self.agents[0].move(query_dict))
            #print(f'{self.agents[0].name} plays {desired_move}')
            # if the move is invalid, select random valid move
            if desired_move not in temp_valid_moves:
                random_index = np.random.choice(np.arange(len(temp_valid_moves)))
                desired_move = tuple(temp_valid_moves[random_index])
                print('playing invalid move: converting to random valid')

            # update board and
            self.game_log.append(desired_move)
            self.move(position = desired_move)
            
        except:
            # shouldn't get here, but this chunk of code exists for safety
            print(f'warning! exception raised in "query" for {self.agents[0].name}')
            random_index = np.random.choice(np.arange(len(temp_valid_moves)))
            desired_move = tuple(temp_valid_moves[random_index])

            # update board and
            self.game_log.append(desired_move)
            self.move(position = desired_move)
            
    def load_agents(self, agent1: bot_template, agent2: bot_template) -> None:
        ''' agent1 and agent2 are uttt agents '''
        self.agents = [agent1, agent2]
        
    def switch_active_player(self) -> None:
        ''' switch the current player value and the agent list '''
        # this is called at the end of .move()
        self.agents = self.agents[::-1]
        self.current_player *= -1
        
    def getwinner(self) -> int:
        ''' new method
        returns the integer indicating the winning player
        (subject to change)
        '''
        if self.finished:
            if self.finished_win:
                return self.current_player
            else:
                return 0
    
    def check_validity(self, position: tuple) -> bool:
        ''' check whether position - a tuple - is valid '''
        box_validity = (self.active_box == self.map_to_major(position)) \
                        or (self.active_box == (-1,-1))
        open_validity = (self.board_state[position] == 0)
        return box_validity and open_validity
    
    def check_line(self, box: np.array) -> bool:
        '''
        box is a (3,3) array (typically a mini-board)
        returns True if a line is found
        '''
        for i in range(3):
            if abs(sum(box[:,i])) == 3: return True # horizontal
            if abs(sum(box[i,:])) == 3: return True # vertical

        # diagonals
        if abs(box.trace()) == 3: return True
        if abs(np.rot90(box).trace()) == 3: return True

    def map_to_major(self, position: tuple) -> tuple:
        '''
        converts position to major coordinates
        eg: (5,3) -> (1,1)
        '''
        return(position[0]//3, position[1]//3)

    def map_to_minor(self, position: tuple) -> tuple:
        '''
        converts position into mini coordinates
        eg: (5,3) -> (2,0)
        '''
        return (position[0]%3, position[1]%3)

    def check_full_stale(self) -> None:
        ''' this might be impossible? '''
        # get number of invalid boxes
        
        if (self.finished_boxes == 0).sum() == 0:
            self.finished_stale = True
            self.finished = True

    def move(self, position: tuple) -> None:
        '''
        the main game logic. board updates and logic checks.
        '''
        if self.finished:
            print('no move played, game is finished')
            return
        
        if self.check_validity(position):
            
            # place marker
            self.board_state[position] = self.current_player
            
            # select both scales
            temp_box = self.map_to_major(position)
            temp_minor_box = self.board_state[3*temp_box[0]:3*temp_box[0]+3,
                                              3*temp_box[1]:3*temp_box[1]+3]
            
            ''' check line at minor scale '''
            if self.check_line(temp_minor_box):
                self.finished_boxes[self.map_to_major(position)] = self.current_player
                
                # check line at major scale
                if self.check_line(self.finished_boxes):
                    self.finished_win = True
                    self.finished = True
                    return # end the whole thing immediately (will cause stalemate bug without this !)

            # if no squares are open, mark as stale
            elif (temp_minor_box == 0).sum() == 0:
                self.finished_boxes[self.map_to_major(position)] = 6 # indicates stalemate in that box
            
            ''' is the whole game board stale? '''
            # if it's stale, set the appropriate flags
            self.check_full_stale()
            
            ''' calculate active box '''
            self.active_box = self.map_to_minor(position)
            # if that box is won or stale flag it
            if self.finished_boxes[self.active_box] != 0:
                self.active_box = (-1,-1)

            # switch player
            self.switch_active_player()

    def get_valid_moves(self) -> np.array:
        '''
        returns an array (N,2) of valid moves
        '''
        
        if self.finished:
            print('no valid moves in terminal state')
            return np.empty(0)
        # define masks that cover the board
        # across the whole board
        full_board_mask = (self.board_state == 0)
        # active square
        active_box_mask = np.zeros((9,9),dtype=bool)
        # identifies finished major boxes
        a = np.repeat(self.finished_boxes,3).reshape(3,9)
        b = np.tile(a,3).reshape(9,9)
        finished_box_mask = (b == 0)
        
        if self.active_box == (-1,-1):
            active_box_mask[:] = True
            active_box_mask *= finished_box_mask
        else:
            active_box_mask[3*self.active_box[0]:3*self.active_box[0]+3,
                            3*self.active_box[1]:3*self.active_box[1]+3] = True

        # return get union of maps
        return np.array(np.where(active_box_mask * full_board_mask)).T

    def draw_valid_moves(self) -> None:
        ''' visualization tool
        plots the valid moves as purple squares
        to be called after the .draw_board() method
        '''
        moves = self.get_valid_moves()
        plt.scatter(moves[:,0],moves[:,1],marker='s',c='purple',alpha=0.3, s=50)
        
    def draw_board(self, marker_size: int = 100) -> None:
        ''' visualization tool
        plots a checkerboard and markers for all plays.
        lines distinguish mini-boards and finished boards are coloured in
        '''
        plt.imshow(checkerboard((9,9)), cmap=cmap, origin='lower')
        for i in [-0.5,2.5,5.5, 8.5]:
            plt.axvline(i,c='k')
            plt.axhline(i,c='k')
        plt.axis('off')

        plt.scatter(*np.where(self.board_state == -1), marker='x', s=marker_size, c='tab:blue')
        plt.scatter(*np.where(self.board_state == 1),  marker='o', s=marker_size, c='tab:orange')
        
        x_boxes = np.where(self.finished_boxes == -1)
        o_boxes = np.where(self.finished_boxes == 1)
        plt.scatter(x_boxes[0]*3+1,x_boxes[1]*3+1,marker='s',s=marker_size*50,alpha=0.6,c='tab:blue')
        plt.scatter(o_boxes[0]*3+1,o_boxes[1]*3+1,marker='s',s=marker_size*50,alpha=0.6,c='tab:orange')
        
        stale_boxes = np.where(self.finished_boxes == 6)
        plt.scatter(stale_boxes[0]*3+1, stale_boxes[1]*3+1, marker='s', s=marker_size*50, alpha=0.3,c='k')



### bot and examples

For those not familiar with inheritance, when you define your bot in a class, you can call the "bot template" as an argument. This will copy all functions from the "parent" to your new class. You can use this to build up a heirarchy of functions (to keep your bot tidy), but it also helps ensure the organizers 

Using inheritence is not necessary.

In [34]:
class error_bot(bot_template):
    ''' for internal development
    returns invalid move (the engine will automatically play a random valid move)
    '''
    def move(self, board_dict):
        return 'lower left please'
    
class random_bot(bot_template):
    '''
    selects random valid move (the default action)
    '''
    def move(self, board_dict):
        random_index = np.random.choice(len(board_dict['valid_moves']))
        return board_dict['valid_moves'][random_index]

class first_bot(bot_template):
    '''
    plays the first valid move
    '''
    def move(self, board_dict):
        return board_dict['valid_moves'][0]


In [35]:
class finns_heuristic_bot(bot_template):
    '''
    this could use a significant refactor
    this provides a lot of methods to pull and remix
    really doesn't use valid_moves
    (valid moves can be determined from board_state)
    
    broadly: this bot tries to block the opponent, take the center,
    take a corner, then takes an edge
    
    it's not very strong, but a useful milestone.
    '''
    
    def _check_line(self, box: np.array) -> bool:
        '''
        box is a (3,3) array
        returns True if a line is found, else returns False '''
        for i in range(3):
            if abs(sum(box[:,i])) == 3: return True # horizontal
            if abs(sum(box[i,:])) == 3: return True # vertical

        # diagonals
        if abs(box.trace()) == 3: return True
        if abs(np.rot90(box).trace()) == 3: return True
        return False

    def _check_line_playerwise(self, box: np.array, player: int = None):
        ''' returns true if the given player has a line in the box, else false
        if no player is given, it checks for whether any player has a line in the box'''
        if player == None:
            return self._check_line(box)
        if player == -1:
            box = box * -1
        box = np.clip(box,0,1)
        return self._check_line(box)
    
    def pull_mini_board(self, board_state: np.array, mini_board_index: tuple) -> np.array:
        ''' extracts a mini board from the 9x9 given the its index'''
        temp = board_state[mini_board_index[0]*3:(mini_board_index[0]+1)*3,
                           mini_board_index[1]*3:(mini_board_index[1]+1)*3]
        return temp

    def get_valid(self, mini_board: np.array) -> np.array:
        ''' gets valid moves in the miniboard'''
        return np.where(mini_board == 0)

    def block_imminent(self, mini_board: np.array) -> list:
        ''' tries to block the opponent if they have 2/3rds of a line '''
        # loop through valid moves with enemy position there.
        # if it makes a line it's imminent
        imminent = list()

        for _valid in zip(*self.get_valid(mini_board)):
            # create temp valid pattern
            valid_filter = np.zeros((3,3))
            valid_filter[_valid[0],_valid[1]] = 1
            if self._check_line(mini_board + valid_filter):
                imminent.append(_valid)
        return imminent

    def get_finished(self, board_state: np.array) -> np.array:
        ''' calculates the completed boxes'''
        opp_boxes = np.zeros((3,3))
        self_boxes = np.zeros((3,3))
        stale_boxes = np.zeros((3,3))
        # look at each miniboard separately
        for _r in range(3):
            for _c in range(3):
                temp_miniboard = self.pull_mini_board(board_state, (_r,_c))
                self_boxes[_r,_c] = self._check_line_playerwise(temp_miniboard, player = 1)
                opp_boxes[_r,_c] = self._check_line_playerwise(temp_miniboard, player = -1)
                if sum(abs(temp_miniboard.flatten())) == 9:
                    stale_boxes[_r,_c] = 1                   

        # return finished boxes (separated by their content)
        return (opp_boxes*-1, self_boxes, stale_boxes)

    def heuristic_mini_to_major(self, board_state: np.array,
                                active_box: tuple,
                                valid_moves: list) -> tuple:
        ''' main function
        block opponent, try to take a corner, else random
        '''
        # if we don't provide active box, we'd have to infer it
        if active_box != (-1,-1):
            # look just at the mini board
            temp_miniboard = self.pull_mini_board(board_state, active_box)
            # look using the logic, select a move
            move = self.mid_heuristic(temp_miniboard)
            # project back to original board space
            return (move[0] + 3 * active_box[0],
                    move[1] + 3 * active_box[1])

        else:
            # use heuristic on finished boxes to select which box to play in
            imposed_active_box = self.major_heuristic(board_state)

            # call this function with the self-imposed active box
            return self.heuristic_mini_to_major(board_state = board_state,
                                           active_box = imposed_active_box,
                                           valid_moves = valid_moves)

    def major_heuristic(self, board_state: np.array) -> tuple:
        ''' determines which miniboard to play on'''
        z = self.get_finished(board_state)
        # finished boxes is a tuple of 3 masks: self, opponent, stale 
        self_boxes  = z[0]
        opp_boxes   = z[1]
        stale_boxes = z[2]
        
        # identify imminent wins
        imminent_wins = self.block_imminent(self_boxes + opp_boxes)
        
        # make new list to remove imminent wins that point to stale boxes
        stale_boxes = list(zip(*np.where(stale_boxes)))
        for stale_box in stale_boxes:
            if stale_box in imminent_wins:
                imminent_wins.remove(stale_box)
        if len(imminent_wins) > 0:
            return imminent_wins[np.random.choice(len(imminent_wins))]

        # take center if available
        internal_valid = list(zip(*self.get_valid(self_boxes + opp_boxes)))
        for stale_box in stale_boxes:
            if stale_box in internal_valid:
                internal_valid.remove(stale_box)

        if (1,1) in internal_valid:
            return (1,1)

        # else take random corner
        _corners = [(0,0),(0,2),(0,2),(2,2)]
        _valid_corner = list()

        for _corner in _corners:
            if _corner in internal_valid:
                _valid_corner.append(_corner)
        if len(_valid_corner) > 0:
            return _valid_corner[np.random.choice(len(_valid_corner))]

        # else take random
        return internal_valid[np.random.choice(len(internal_valid))]
        
    def mid_heuristic(self, miniboard: np.array) -> tuple:
        ''' main logic '''
        # block imminent wins on this miniboard
        imminent_wins = self.block_imminent(miniboard)
        if len(imminent_wins) > 0:
            return imminent_wins[np.random.choice(len(imminent_wins))]

        # take center if available
        internal_valid = list(zip(*self.get_valid(miniboard)))
        if (1,1) in internal_valid:
            return (1,1)

        # else take random corner
        _corners = [(0,0),(0,2),(0,2),(2,2)]
        _valid_corner = list()

        for _corner in _corners:
            if _corner in internal_valid:
                _valid_corner.append(_corner)
        if len(_valid_corner) > 0:
            return _valid_corner[np.random.choice(len(_valid_corner))] # must convert back to full board tuple

        # else take random
        return internal_valid[np.random.choice(len(internal_valid))]

    def move(self, board_dict: dict) -> tuple:
        ''' wrapper
        apply the logic and returns the desired move
        '''
        return tuple(self.heuristic_mini_to_major(board_state = board_dict['board_state'],
                                                  active_box = board_dict['active_box'],
                                                  valid_moves = board_dict['valid_moves']))


### Bot testing


In [58]:
def initialize(engine, n_moves:int) -> None:
    if n_moves%2 != 0: print('warning: number of moves should be even!')
    
    for i in range(n_moves):
        valid_moves = engine.get_valid_moves()
        random_index = np.random.choice(len(valid_moves))
        engine.move(tuple(valid_moves[random_index]))

def run_many_games(agent1: bot_template,
                   agent2: bot_template,
                   n_games: int = 10,
                   n_init_moves: int = 4):
    wins = list()
    for i in tqdm(range(n_games)):
        finished_flag = False
        engine = uttt_engine_agentic_refactor()
        engine.load_agents(agent1, agent2)
        initialize(engine, n_moves=n_init_moves)
        while not finished_flag:
            engine.query_player()
            if engine.finished:
                finished_flag = True
        wins.append(engine.getwinner())
        
    # return stats
    if sum(wins) > 0: print(agent1.name, 'is the overall winner')
    if sum(wins) < 0: print(agent2.name, 'is the overall winner')
    if sum(wins) == 0: print(agent1.name,'and',agent2.name,'are evenly matched')
    return np.array(wins)

In [59]:
''' core imports '''
import numpy as np
import matplotlib.pyplot as plt
import queue
from bigtree import Node

''' development imports'''
from time import perf_counter
from tqdm import tqdm

''' visualization imports '''
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('mycmap', ['lightgrey', 'white'])
import matplotlib.colors as mcolors
tab10_names = list(mcolors.TABLEAU_COLORS) # create a list of colours


# author: Aidan
# date: Nov 28 2023
# notes: Using line_completer.py as the baseline then added my own stuff in
# will look into using a tree search
# https://michaelxing.com/UltimateTTT/v3/ai/ bot seems to try and get two in a row for squares it is tring to get
# for squares it is not trying to get it puts them in bad squares to stop me from getting the square while also trying
# to land me in a bad sqaure to play in. 
# the AI seems to think the middle square in the middle is the best first move
# The best response to that is to put the other player into the corner square
# appears to be a good strategy to not repeat squares until you have to as it can allow the oponent to get two in a row
# dont make a move that will result in oponent putting you right back where you were when you dont want to be there
# all about wanting oponent to make forcing moves. Or a good move for them results in a better move for you
# maybe make a bot that tries to make a move that forces the opponent to make a move that is bad for them rather
# than making a move that is good for you
# the 9 major squares some or more valuable then others. Winning those squares is a priority
# The squares are ranked at start and can change based on game state

class BotKiller:
    #BotKiller
    MPQueue = queue.PriorityQueue() #put values in negative to get max
    root = Node
    #key is value given and the actual value is the move
    ''' ------------------ required function ---------------- '''
    
    def __init__(self,name: str = 'Chekhov') -> None:
        self.name = name
        # define the probability distribution
        self.box_probs = np.ones((3,3)) # edges
        self.box_probs[1,1] = 4 # center
        self.box_probs[0,0] = self.box_probs[0,2] = self.box_probs[2,0] = self.box_probs[2,2] = 2 # corners
        self.MajorSquareRank = np.array([[2,1,2],[1,3,1],[2,1,2]])#middle most valuable, then corners then edges
        self.InitialMajorSquareRank = np.array([[2,1,2],[1,3,1],[2,1,2]])#middle most valuable, then corners then edges
        #because 4 moves are made before start may be worth anaylizing the board and changing the rank of the squares
        self.GameState = np.zeros((3,3))#State of major boxes. (i.e. the total game state of the board)
    
    def move(self, board_dict: dict) -> tuple:
        ''' wrapper
        apply the logic and returns the desired move
        The tuple being returned is in reference to the 81 squares, so it could be anything from (0,0) to 
        (8,8) - x column then y row. This avoids ambiguity when a player can play in multiple miniboards 
        (such as when their opponent sends them to a completed miniboard).
        '''
        
        return tuple(self.ChooseBestMajorSquare(board_state = board_dict['board_state'],
                                                  active_box = board_dict['active_box'],
                                                  valid_moves = board_dict['valid_moves']))
    ''' --------- generally useful bot functions ------------ '''
    
    def _check_line(self, box: np.array) -> bool:
        '''
        box is a (3,3) array
        returns True if a line is found, else returns False '''
        for i in range(3):
            if abs(sum(box[:,i])) == 3: return True # horizontal
            if abs(sum(box[i,:])) == 3: return True # vertical

        # diagonals
        if abs(box.trace()) == 3: return True
        if abs(np.rot90(box).trace()) == 3: return True
        return False

    def _check_line_playerwise(self, box: np.array, player: int = None):
        ''' returns true if the given player has a line in the box, else false
        if no player is given, it checks for whether any player has a line in the box'''
        if player == None:
            return self._check_line(box)
        if player == -1:
            box = box * -1
        box = np.clip(box,0,1)
        return self._check_line(box)
    
    def pull_mini_board(self, board_state: np.array, mini_board_index: tuple) -> np.array:
        ''' extracts a mini board from the 9x9 given its index'''
        temp = board_state[mini_board_index[0]*3:(mini_board_index[0]+1)*3,
                           mini_board_index[1]*3:(mini_board_index[1]+1)*3]
        return temp

    def get_valid(self, mini_board: np.array) -> np.array:
        ''' gets valid moves in the miniboard'''
        return np.where(mini_board == 0)

    def get_finished(self, board_state: np.array) -> np.array:
        ''' calculates the completed boxes'''
        opp_boxes = np.zeros((3,3))
        self_boxes = np.zeros((3,3))
        stale_boxes = np.zeros((3,3))
        # look at each miniboard separately
        for _r in range(3):
            for _c in range(3):
                mini_board = self.pull_mini_board(board_state, (_r,_c))
                self_boxes[_r,_c] = self._check_line_playerwise(mini_board, player = 1)
                opp_boxes[_r,_c] = self._check_line_playerwise(mini_board, player = -1)
                if sum(abs(mini_board.flatten())) == 9:
                    stale_boxes[_r,_c] = 1                   

        # return finished boxes (separated by their content)
        return (opp_boxes*-1, self_boxes, stale_boxes)
    
    def get_num_finished(self, board_state: np.array):
        ''' returns a list of the number of finished boxes for each player'''
        Boxes = [0,0,0]
        for r in range(3):
            for c in range(3):
                if(board_state[r,c] == 1):
                    Boxes[0]+=1
                elif(board_state[r,c] == -1):
                    Boxes[1]+=1
                else:
                    Boxes[2]+=1
        return Boxes

    def complete_line(self, mini_board: np.array) -> list:
        ''' completes a line if available '''
        # loop through valid moves with hypothetic self position there.
        # if it makes a line it's an imminent win
        imminent = list()
        valid_moves = self.get_valid(mini_board)
        
        for _valid in zip(*valid_moves):
            # create temp valid pattern
            valid_filter = np.zeros((3,3))
            valid_filter[_valid[0],_valid[1]] = 1
            if self._check_line(mini_board + valid_filter):
                imminent.append(_valid)
        return imminent
    
    def get_probs(self, valid_moves: list) -> np.array:
        ''' match the probability with the valid moves to weight the random choice '''
        probs = list()
        for _valid in valid_moves:
            probs.append(self.box_probs[_valid[0],_valid[1]])
        probs /= sum(probs) # normalize
        return probs
    
    ''' ------------------ bot specific logic ---------------- '''
    
    def ChooseBestMajorSquare (self, board_state: np.array, active_box: tuple, valid_moves: list):
        ''' chooses the best major square to play in based on the current board state'''
        # if the active box is not the whole board
        if active_box != (-1,-1):
            # look just at the mini board
            mini_board = self.pull_mini_board(board_state, active_box)
            # look using the logic, select a move
            move = self.ChooseBestMiniSquare(mini_board)
            # project back to original board space
            return (move[0] + 3 * active_box[0],
                    move[1] + 3 * active_box[1])

        else:
            # use heuristic on finished boxes to select which box to play in
            imposed_active_box = self.CalculateBoardState(board_state)

            # call this function with the self-imposed active box
            return self.ChooseBestMajorSquare(board_state = board_state,
                                                active_box = imposed_active_box,
                                                valid_moves = valid_moves)
        
    def CalculateBoardState(self, board_state: np.array):
        ''' calculates the current state of the board for Major Square Heuristic'''
        for i in range(3):
            for j in range(3):
                self.GameState[i,j] = self.CalculateMiniBoardState(self.pull_mini_board(board_state, (i,j)))
                '''if (self.GameState[i,j] == 1):
                    self.MajorSquareRank[i,j] = 1
                elif(self.GameState[i,j] == -1):
                    self.MajorSquareRank[i,j] = -1
                elif(self.GameState[i,j] == 0):
                    self.MajorSquareRank[i,j] = 0
                else:
                    self.MajorSquareRank[i,j]+=self.GameState[i,j]#need to test this. See if its any good'''
                    #or do the following instead. GameState is for board state, MSR is for value of major squares
                if (self.GameState[i,j] == 1 or self.GameState[i,j] == -1 or self.GameState[i,j] == 0):
                    #If a Major square is won, lost or stalemate then it is worth 0
                    self.MajorSquareRank[i,j] = 0
                else:
                    #If a Major square is not won, lost or stalemate then it is worth the value of the mini squares * InitialMajorSquareRank
                    self.MajorSquareRank[i,j] = self.GameState[i,j]*self.InitialMajorSquareRank[i,j]

    def CalculateMiniBoardState(self, mini_board: np.array):
        ''' calculates the current state of the mini board for ChooseBestMiniSquare'''
        #if the board is won return 1
        if self._check_line_playerwise(mini_board, player = 1):
            return 1
        #if the board is lost return -1
        if self._check_line_playerwise(mini_board, player = -1):
            return -1
        #if the board has more opponent moves in it return -0.5
        get_finished = self.get_num_finished(mini_board)
        #[0] is self, [1] is opponent, [2] is stale
        if get_finished[1]>get_finished[0]:
            return -0.5
        #if the board has more self moves in it return 0.5
        if(get_finished[1]<get_finished[0]):
            return 0.5
        #if the board is a stalemate return 0
        if(get_finished[2] == 9):
            return 0
        #if the board has same amount of opponent moves as self moves return 0.1
        return 0.1
        
    def CalculateMiniBoardStateEnemy(self, mini_board: np.array):
        ''' calculates the current state of the mini board for ChooseBestMiniSquare'''
        #if the board is won return 1
        if self._check_line_playerwise(mini_board, player = -1):
            return 1
        #if the board is lost return -1
        if self._check_line_playerwise(mini_board, player = 1):
            return -1
        #if the board has more opponent moves in it return -0.5
        get_finished = self.get_num_finished(mini_board)
        #[0] is self, [1] is opponent, [2] is stale
        if get_finished[1]>get_finished[0]:
            return -0.5
        #if the board has more self moves in it return 0.5
        if(get_finished[1]<get_finished[0]):
            return 0.5
        #if the board is a stalemate return 0
        if(get_finished[2] == 9):
            return 0
        #if the board has same amount of opponent moves as self moves return 0.1
        return 0.1

    def MoveOutcome(self, board_state: np.array, move: tuple):
        ''' calculates the outcome of a move'''
        temp_board = board_state
        temp_board[move[0],move[1]] = 1
        return self.CalculateMiniBoardStateEnemy(temp_board)


    def ChooseBestMiniSquare (self, miniBoard: np.array):
        #look through the mini board and find the best square to play in based on what it does for me.
        #first check if can make a move that wins me a Major square. Then if it does check to see if winning
        #that square is valuable or not using CalculateMiniBoardState. If it is valuable play there. If not put in MP Queue
        #then check other possible moves and see what major square it puts them into. Check the value of those squares
        #using CalculateMiniBoardState. If it is valuable dont play there. Want to play a move that puts the opponent
        #in a bad major square. For each, put into the MPQueue. Once all moves are checked, take the move with the
        #highest value from the MPQueue and play it. 
        Best_Move = [0,0]
        for i in range(3):
                for j in range (3):
                    #print(miniBoard)
                    if miniBoard[i,j] == 0:
                        temp_board = miniBoard
                        temp_board[i,j] = 1
                        MiniBoardState = self.CalculateMiniBoardState(temp_board)
                        MiniBoardState+=self.MoveOutcome(miniBoard, (i,j))
                        self.MPQueue.put((-MiniBoardState, i,j))#put in negative to get max
                        #check if it wins me a major square
                        #if it does, check if that square is valuable
                        #if it is, play there
                        #if not, put in MPQueue
                        #check other possible moves and see what major square it puts them into
                        #check the value of those squares
                        #if it is valuable dont play there
                        #want to play a move that puts the opponent in a bad major square
                        #for each, put into the MPQueue
                        #once all moves are checked, take the move with the highest value from the MPQueue and play it
                    
        #return the move with the highest value from the MPQueue
        Best_Move[0] = self.MPQueue.get()[1]
        Best_Move[1] = self.MPQueue.get()[2]
        self.MPQueue.queue.clear()
        return Best_Move#Temp




In [65]:
agent1 = BotKiller(name = 'marty')
agent2 = finns_heuristic_bot(name = 'Aljoscha')

stats = run_many_games(agent1 = agent1, agent2 = agent2)

  0%|          | 0/100 [07:45<?, ?it/s]


KeyboardInterrupt: 

In [64]:
print(f'{agent1.name} wins:', sum(stats == 1))
print(f'{agent2.name} wins:', sum(stats == -1))
print('draws:', sum(stats == 0))

marty wins: 105
Aljoscha wins: 834
draws: 61
